In [2]:
import pandas as pd
import numpy as np
import gymnasium as gym
from tqdm import tqdm

In [3]:
env = gym.make(id='MountainCar-v0')

In [4]:
pos = np.linspace(env.observation_space.low[0], env.observation_space.high[0],20)
vel = np.linspace(env.observation_space.low[1], env.observation_space.high[1],20)

In [5]:
q_table = np.zeros((len(pos), len(vel), env.action_space.n))
ntrained=0

In [6]:
episodes = 5000
epsilon = 1
epsilon_decay = 2/episodes
learning_rate = 0.8
discount_factor = 0.8
rewardEpisodes = np.zeros(episodes)

In [7]:
for i in tqdm(range(episodes), colour='GREEN', desc='Learning', unit=' Episodes'):
    state=env.reset()[0]
    p_state = np.digitize(state[0], pos)
    v_state = np.digitize(state[1], vel)
    terminated = False
    rewards=0
    while (not terminated  and rewards>-1000):
        if np.random.random() < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[p_state, v_state])
        newstate,reward,terminated,_,_ = env.step(action)
        new_p_state = np.digitize(newstate[0], pos)
        new_v_state = np.digitize(newstate[1], vel)
        q_table[p_state, v_state, action] = (1-learning_rate)*q_table[p_state, v_state, action] + learning_rate*(
            reward + discount_factor*np.max(q_table[ new_p_state, new_v_state]))
        p_state, v_state = new_p_state, new_v_state
        rewards+=reward
        
    epsilon = max(epsilon - epsilon_decay,0)   
    rewardEpisodes[i] = 1 
    if epsilon == 0:
        learning_rate = 0.0001
# env.close()        
        

Learning: 100%|█████████████████████████████████████████████████████████████| 5000/5000 [02:38<00:00, 31.65 Episodes/s]


In [8]:
env = gym.make(id='MountainCar-v0', render_mode='human')

In [9]:
for i in range(10):
    state=env.reset()[0]
    p_state = np.digitize(state[0], pos)
    v_state = np.digitize(state[1], vel)
    terminated = False
    while (not terminated):
        action = np.argmax(q_table[p_state, v_state])
        newstate,reward,terminated,_,_ = env.step(action)
        p_state = np.digitize(newstate[0], pos)
        v_state = np.digitize(newstate[1], vel)
env.close()